In [3]:
from datasets import load_dataset

In [4]:
dataset=load_dataset('imdb')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [6]:
from datasets import DatasetDict

In [7]:
dataset=DatasetDict(
    {
        'train':dataset['train'],
        'test':dataset['test']
    }
)

In [8]:
dataset['train'][43]

{'text': 'As a kid I did think the weapon the murderer wielded was cool, however I was a kid and so I was a bit dumb. Even as a dumb kid though the movies plot was stupid and a bit boring when the killer was not using his light knife to kill people. What amazes me is that the movie has a really solid cast in it. What script did they read when agreeing to be in this movie as it is most assuredly boring and only a means to show off a light saber on a very small scale. The plot at times is incomprehensible and the end is totally chaotic. The whole film seems to rotate around aliens and the one weapon. The plot has two kids and some dude having an alien encounter, flash years later and there seems to be a return as it were in the mix. Dead animals and such to be explored and for some reason the one dude gets the weapon of the aliens and proceeds to use it to go on a very light killing spree. Seriously, you just have to wonder why this movie was made, if you are going to have a killer have 

In [9]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification

In [10]:
tok=AutoTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [11]:
def tok_fn(batch):
  return tok(
    list(batch['text']),
    truncation=True,
    max_length=512
)

In [12]:
tok_data=dataset.map(tok_fn,batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [13]:
from transformers import DataCollatorWithPadding

In [14]:
collat=DataCollatorWithPadding(tokenizer=tok)

In [15]:
from transformers import TrainingArguments,Trainer

In [16]:
from sklearn.metrics import f1_score, accuracy_score
import torch

In [17]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), axis=-1)
    acc = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average="macro")
    return {"accuracy": acc, "macro_f1": macro_f1}

In [18]:
seeds = [42, 123, 999]
trainers = []

In [19]:
for seed in seeds:
    model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
    training_args = TrainingArguments(
        output_dir=f"bert_seed{seed}",
        num_train_epochs=2,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        seed=seed,
        load_best_model_at_end=True,
        fp16=True,
        save_total_limit=2,
        metric_for_best_model="accuracy"
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tok_data["train"],
        eval_dataset=tok_data["test"],
        tokenizer=tok,
        data_collator=collat,
        compute_metrics=compute_metrics
    )
    trainer.train()
    trainers.append(trainer)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1612326234.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bhalaniakshat (bhalaniakshat-dwarkadas-j-sanghvi-college-of-engineering) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,0.328200,0.359540,0.899800,0.899274
2,0.155900,0.276046,0.937400,0.937400


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1612326234.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,0.346300,0.219419,0.924960,0.924916


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,0.346300,0.219419,0.924960,0.924916
2,0.180300,0.261808,0.938360,0.938360


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1612326234.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,0.323400,0.237565,0.929400,0.929398
2,0.151700,0.265952,0.939800,0.939800


In [22]:
from huggingface_hub import login
login()

In [23]:
for i, trainer in enumerate(trainers):
    trainer.save_model(f"./bert_imdb_ensemble/model_{i}")
tok.save_pretrained("./bert_imdb_ensemble/tokenizer")

('./bert_imdb_ensemble/tokenizer/tokenizer_config.json',
 './bert_imdb_ensemble/tokenizer/special_tokens_map.json',
 './bert_imdb_ensemble/tokenizer/vocab.txt',
 './bert_imdb_ensemble/tokenizer/added_tokens.json',
 './bert_imdb_ensemble/tokenizer/tokenizer.json')

In [24]:
models=[]
for i in range(3):
  model=AutoModelForSequenceClassification.from_pretrained(f'./bert_imdb_ensemble/model_{i}')
  model.eval()
  models.append(model)
tok=AutoTokenizer.from_pretrained('./bert_imdb_ensemble/tokenizer')

In [30]:
def ensemble_predict(trainers, texts, weights=None):
    device = next(trainers[0].model.parameters()).device
    inputs = tok(texts, truncation=True, padding=True, max_length=512, return_tensors="pt").to(device)
    logits_list = []
    for trainer in trainers:
        trainer.model.to(device)
        trainer.model.eval()
        with torch.no_grad():
            outputs = trainer.model(**inputs)
            logits_list.append(outputs.logits)

    logits_stack = torch.stack(logits_list)

    if weights is not None:
        weights_tensor = torch.tensor(weights,device=device).view(-1,1,1)
        weighted_logits = (logits_stack * weights_tensor).sum(dim=0)
    else:
        weighted_logits = logits_stack.mean(dim=0)

    probs = torch.nn.functional.softmax(weighted_logits, dim=-1)
    preds = torch.argmax(probs, dim=-1)
    return preds.cpu(), probs.cpu()


In [31]:
from huggingface_hub import upload_folder
upload_folder(
    folder_path='./bert_imdb_ensemble',
    repo_id='ByteMeHarder-404/bert-imdb-ensemble',
    repo_type='model'
)

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ..._ensemble/model_2/training_args.bin: 100%|##########| 5.78kB / 5.78kB            

  ..._ensemble/model_1/training_args.bin: 100%|##########| 5.78kB / 5.78kB            

  ..._ensemble/model_0/training_args.bin: 100%|##########| 5.78kB / 5.78kB            

  ..._ensemble/model_1/model.safetensors:   2%|1         | 8.38MB /  438MB            

  ..._ensemble/model_2/model.safetensors:   2%|1         | 8.36MB /  438MB            

  ..._ensemble/model_0/model.safetensors:   2%|1         | 8.29MB /  438MB            

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/ByteMeHarder-404/bert-imdb-ensemble/commit/9c4e6c0c57fe8f452203c53c0c294379339d3815', commit_message='Upload folder using huggingface_hub', commit_description='', oid='9c4e6c0c57fe8f452203c53c0c294379339d3815', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ByteMeHarder-404/bert-imdb-ensemble', endpoint='https://huggingface.co', repo_type='model', repo_id='ByteMeHarder-404/bert-imdb-ensemble'), pr_revision=None, pr_num=None)

In [32]:
reviews = [
    "This movie was an absolute masterpiece!",
    "I got bored halfway through, not recommended.",
    "It was okay, some good parts but mostly boring."
]

preds, probs = ensemble_predict(trainers, reviews,[0.2,0.2,0.6])
for r, p, prob in zip(reviews, preds, probs):
    print(f"\nReview: {r}")
    print(f"Prediction: {p.item()} (POSITIVE=1 / NEGATIVE=0)")
    print(f"Probabilities: {prob.tolist()}")


Review: This movie was an absolute masterpiece!
Prediction: 1 (POSITIVE=1 / NEGATIVE=0)
Probabilities: [0.0009007977787405252, 0.9990992546081543]

Review: I got bored halfway through, not recommended.
Prediction: 0 (POSITIVE=1 / NEGATIVE=0)
Probabilities: [0.9992891550064087, 0.0007107790443114936]

Review: It was okay, some good parts but mostly boring.
Prediction: 0 (POSITIVE=1 / NEGATIVE=0)
Probabilities: [0.9992732405662537, 0.000726770784240216]
